# Day 3: Gear Ratios
## Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the *water source*, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can *add up all the part numbers* in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently *any number adjacent to a symbol*, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


In [87]:
$sample_data = @'
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
'@ -split "`n"

$data = Get-Content .\data.txt

## Code
I get the feeling that AoC wants us to learn Regex....

But first, let's characterize the actual part symbols used in the full dataset:

In [47]:


$unique_chars = $data -join '' -replace '[0-9.]' -split '' | Where-Object { ![string]::IsNullOrWhiteSpace($_) } | Sort-Object -Unique
Write-Output "Found $($unique_chars.Count) uniques; they were @('$($unique_chars -join "', '")')"


Found 10 uniques; they were @('-', '@', '*', '/', '&', '#', '%', '+', '=', '$')


In [99]:
function dostuff {
    Param(
        [Parameter(Mandatory=$true, Position=0)]
        $data,
        [Parameter(Mandatory=$false, Position=1)]
        [switch] $part2
    )

    $part_symbols = @('-', '@', '*', '/', '&', '#', '%', '+', '=', '$')
    $pattern = "\" + ($part_symbols -join '|\')
    $part_numbers = @()

    $previous_parts = @() #conveniently matches nil for the first line where obvs there's no prior line
    $previous_numbers = @()
    foreach($line in $data) {

        #Gather the parts & numbers on this line
        $parts = [regex]::Matches($line, $pattern)
        $numbers = [regex]::Matches($line, '\d+')
        Write-Verbose ("Found $($parts.Count) parts, at @($($parts.Groups.index -join ", ")) and " + 
                      "$($numbers.Count) part numbers, at @($($numbers.Groups.index -join ", "))")
        
        #reset these to avoid adding 2x
        $part_numbers_current = @()
        $part_numbers_previous = @()
        $part_numbers_next = @()

        #check the previous line's numbers with this line's parts
        if($parts.count -gt 0 -and $previous_numbers.Count -gt 0) {
            $part_numbers_next = $previous_numbers.Groups | Where-Object {
                    (Compare-Object $parts.Groups.Index (($_.index - 1)..($_.index + $_.length)) -ExcludeDifferent -IncludeEqual).Count -gt 0
                    #                   |                   |                   |                       |-------------|---> only return matching numbers (positions)
                    #                   |                   |                   |--> one pos right of the end (remember lenght is *not* 0 indexed)
                    #                   |                   |--> One pos left of the start
                    #                   |                 _____________________________________
                    #                   |                               |---> all the integers from one left of the start to one right of the end
                    #                   |--> array of all the indexes of part symbols
                }
            Write-Verbose "Checking all around numbers from the previous line, found $($part_numbers_next.Count) parts, at @($($part_numbers_next.index -join ", ")) with values @($($part_numbers_next.Value -join ", "))"
        }
        #check this line
        if($parts.count -gt 0) {
            $part_numbers_current = $numbers.Groups | Where-Object {
                    (Compare-Object $parts.Groups.Index (($_.index - 1)..($_.index + $_.length)) -ExcludeDifferent -IncludeEqual).Count -gt 0
                    #                   |                   |                   |                       |-------------|---> only return matching numbers (positions)
                    #                   |                   |                   |--> one pos right of the end (remember lenght is *not* 0 indexed)
                    #                   |                   |--> One pos left of the start
                    #                   |                 _____________________________________
                    #                   |                               |---> all the integers from one left of the start to one right of the end
                    #                   |--> array of all the indexes of part symbols
                }
            Write-Verbose "Checking all around, found $($part_numbers_current.Count) parts, at @($($part_numbers_current.index -join ", ")) with values @($($part_numbers_current.Value -join ", "))"
        }
        #check the previous line's 
        if($previous_parts.Count -gt 0) {
           $part_numbers_previous = $numbers.Groups | Where-Object {
                    (Compare-Object $previous_parts.Groups.Index (($_.index - 1)..($_.index + $_.length)) -ExcludeDifferent -IncludeEqual).Count -gt 0
                }
            Write-Verbose "Checking all around parts from the previous line, found $($part_numbers_previous.Count) parts, at @($($part_numbers_previous.index -join ", ")) with values @($($part_numbers_previous.Value -join ", "))"

        }

        $part_numbers += $part_numbers_current
        $part_numbers += $part_numbers_previous
        $part_numbers += $part_numbers_next

        #Prep for next line
        $previous_parts = $parts
        $previous_numbers = $numbers
    }
    Write-Debug "Found a total of $($part_numbers.Count): @($($part_numbers.value -join ", "))"

    return ($part_numbers.Value | Measure-Object -Sum).Sum

}


VERBOSE: Found 0 parts, at @() and 2 part numbers, at @(0, 5)
VERBOSE: Found 1 parts, at @(3) and 0 part numbers, at @()
VERBOSE: Checking all around numbers from the previous line, found 1 parts, at @(0) with values @(467)
VERBOSE: Checking all around, found 0 parts, at @() with values @()
VERBOSE: Found 0 parts, at @() and 2 part numbers, at @(2, 6)
VERBOSE: Checking all around parts from the previous line, found 1 parts, at @(2) with values @(35)
VERBOSE: Found 1 parts, at @(6) and 0 part numbers, at @()
VERBOSE: Checking all around numbers from the previous line, found 1 parts, at @(6) with values @(633)
VERBOSE: Checking all around, found 0 parts, at @() with values @()
VERBOSE: Found 1 parts, at @(3) and 1 part numbers, at @(0)
VERBOSE: Checking all around, found 1 parts, at @(0) with values @(617)
VERBOSE: Checking all around parts from the previous line, found 0 parts, at @() with values @()
VERBOSE: Found 1 parts, at @(5) and 1 part numbers, at @(7)
VERBOSE: Checking all aroun

In [ ]:
$VerbosePreference = 'Continue'
$DebugPreference = 'Continue'
dostuff $sample_data

In [100]:
$VerbosePreference = 'SilentlyContinue'
$DebugPreference = 'SilentlyContinue'
Write-Host "Part 1 solution is $(dostuff $data)"

Part 1 solution is 551094


## Part Two

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

## Part 2 code
Decided to just do this one separately since I have to refactor the code so heavily...

In [122]:
function dostuff2 {
    Param(
        [Parameter(Mandatory=$true, Position=0)]
        $data
    )

    $total = 0

    $rows_of_ratios = [System.Collections.ArrayList]@()
    foreach($line in $data){
        $numbers = [regex]::Matches($line, '\d+')
        Write-Verbose ("Found $($numbers.Count) gear ratios(s), at @($($numbers.Groups.index -join ", "))")
        $rows_of_ratios.add($numbers) | Out-Null # is noisy otherwise
    }
    Write-Verbose "There are $($rows_of_ratios.Count) rows of ratios."
    $line_num = 0
    foreach($line in $data) {

        #Find all the gears
        $gears = [regex]::Matches($line, "\*")
        Write-Verbose ("Found $($gears.Count) gear(s), at @($($gears.Groups.index -join ", "))")
        
        $from = $line_num - 1
        $to   = $line_num + 1
        # Cover the literal edge cases
        if($line_num -eq 0) {
            $from = 0
        } elseif($line_num -eq $data.Count-1) {
            $to = $line_num 
        } 

        foreach($gear in $gears) {
            $ratios_of_this_gear = $rows_of_ratios[$from..$to].Groups | Where-Object {
                    (Compare-Object $gear.Index (($_.index - 1)..($_.index + $_.length)) -ExcludeDifferent -IncludeEqual).Count -gt 0

                }
            Write-Verbose "Checking all around, found $($ratios_of_this_gear.Count) ratios, at @($($ratios_of_this_gear.index -join ", ")) with values @($($ratios_of_this_gear.Value -join ", "))"

            if($ratios_of_this_gear.Count -eq 2) {
                $ratio = [int]$ratios_of_this_gear[0].value * [int]$ratios_of_this_gear[1].value
                $total += $ratio
                Write-Debug "Valid gear! $($ratios_of_this_gear[0].value) × $($ratios_of_this_gear[1].value) = $ratio; running total is $total"
            }
        }

        $line_num++
    }
    #Write-Debug "Found a total of $($part_numbers.Count): @($($part_numbers.value -join ", "))"

    return $total

}

In [123]:
$VerbosePreference = 'Continue'
$DebugPreference = 'Continue'
Write-Host "Part 2 solution is $(dostuff2 $sample_data)"

VERBOSE: Found 2 gear ratios(s), at @(0, 5)
VERBOSE: Found 0 gear ratios(s), at @()
VERBOSE: Found 2 gear ratios(s), at @(2, 6)
VERBOSE: Found 0 gear ratios(s), at @()
VERBOSE: Found 1 gear ratios(s), at @(0)
VERBOSE: Found 1 gear ratios(s), at @(7)
VERBOSE: Found 1 gear ratios(s), at @(2)
VERBOSE: Found 1 gear ratios(s), at @(6)
VERBOSE: Found 0 gear ratios(s), at @()
VERBOSE: Found 2 gear ratios(s), at @(1, 5)
VERBOSE: There are 10 rows of ratios.
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 1 gear(s), at @(3)
VERBOSE: Checking all around, found 2 ratios, at @(0, 2) with values @(467, 35)
DEBUG: Valid gear! 467 × 35 = 16345; running total is 16345
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 1 gear(s), at @(3)
VERBOSE: Checking all around, found 1 ratios, at @(0) with values @(617)
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 0 gear(s), at @()
VERBOSE: Found 1 gear(s), at @(5)
VERBOSE: Checking all around, fou

In [124]:
$VerbosePreference = 'SilentlyContinue'
$DebugPreference = 'SilentlyContinue'
Write-Host "Part 2 solution is $(dostuff2 $data)"

Part 2 solution is 80179647
